# Imports

In [2]:
from __future__ import print_function


import matplotlib as mpl
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re

from transformers import pipeline

/Users/yasmeenmahomedy2/.pyenv/versions/3.10.6/envs/conflict_prediction/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-07 10:13:52.983548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('/Users/yasmeenmahomedy2/Downloads/Conflict Datasets/un-general-debates.csv')

In [4]:
df2 = df[df['year']>=1990]

In [73]:
#check its in df
df2[df2['country']=='THA']

,session,year,country,text
359,68,2013,THA,On behalf of \nthe delegation of the Kingdom o...
586,63,2008,THA,The attachment \nthat Thailand has had to the ...
897,46,1991,THA,﻿I should like to extend warm congratulations ...
1065,57,2002,THA,"﻿I would like, on\nbehalf of the Government an..."
1379,51,1996,THA,﻿On behalf of the\nGovernment and people of Th...
1572,67,2012,THA,﻿On behalf of the\ndelegation of the Kingdom o...
1834,52,1997,THA,﻿It is with great pleasure\nthat my delegation...
2341,65,2010,THA,"Please allow me, Sir, to \nextend to you, on b..."
2635,50,1995,THA,On behalf of the\nGovernment and people of Tha...
2812,64,2009,THA,On behalf of the \ndelegation of the Kingdom o...


# Sentiment

In [5]:
sentiment = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    tokenizer="distilbert-base-uncased-finetuned-sst-2-english",
)

2022-12-07 10:14:36.217584: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [6]:
def clean_text(text):
    text = text.encode("ascii", errors="ignore").decode(
        "ascii"
    )  # remove non-ascii, Chinese characters
    text = text.lower()
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\t", " ", text)
    text = text.strip(" ")
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation and special characters
    text = re.sub(
        " +", " ", text
    ).strip()  # get rid of multiple spaces and replace with a single
    text = text[:500]
    return text


In [7]:
def sentiment_analysis(text):
    input_text = (
        pd.DataFrame(text.split("."))
        .stack()
        .reset_index()
        .rename(columns={0: "Paras"})
        .drop("level_0", axis=1)
        .drop("level_1", axis=1)
        .dropna()
    )

    input_text["Clean_Text"] = input_text["Paras"].map(lambda text: clean_text(text))

    corpus = list(input_text["Clean_Text"].values)

    input_text["Sentiment"] = sentiment(corpus)

    input_text["Sentiment_Label"] = [x.get("label") for x in input_text["Sentiment"]]

    input_text["Sentiment_Score"] = [x.get("score") for x in input_text["Sentiment"]]

    cols = ["Paras", "Sentiment_Label", "Sentiment_Score"]
    df = input_text[cols].copy()

    df = df[df["Paras"].str.strip().astype(bool)]

    df["Sentiment_Score"] = np.where(
        df["Sentiment_Label"] == "NEGATIVE",
        -(df["Sentiment_Score"]),
        df["Sentiment_Score"],
    )

    df["Sentiment_Score"] = df["Sentiment_Score"].round(6)

    overall_sentiment_score = df["Sentiment_Score"].sum().round(3)

    sentiment_count = df["Sentiment_Label"].value_counts().to_string()

#     return overall_sentiment_score, sentiment_count, df
    return df['Sentiment_Score'].mean()


In [ ]:
#filter out beginning end of speech? on pakistan df
# temp = pak['text'].reset_index()
# speech_0 = temp['text'][1]
# start = round(len(speech_0)*0.1)
# end = round(len(speech_0)*0.9)

# speech_filter = speech_0[start:end]
# sentiment_analysis(speech_filter)

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

# Functions

In [ ]:
#master df 
def sentiment_graph(cc, country):
    df = (master[master['country']==cc])
    fig = go.Figure(
    data=go.Heatmap(
        z=df['Overall_Sentiment_Score'],
        x=df['year'],
        y=df["new_sentiment_score"],
        colorscale=px.colors.sequential.RdBu,
            )
        )

    fig.update_layout(
        title=go.layout.Title(
            text=f'Sentiment Analysis for {country}'
        ),
        autosize=False,
        width=1150,
        height=400,
    )

    fig.update_layout(yaxis_autorange = "reversed")
    return fig


In [48]:
#def to get the countrys df with sentiment stuff init

def get_df(country):
    country_df = df2[df['country']==country]
    country_df['Overall_Sentiment_Score'] = country_df['text'].apply(sentiment_analysis)
    new_df = pd.DataFrame(country_df)
    new_df['new_sentiment_score'] = np.where(new_df['Overall_Sentiment_Score']< 0.3, 'NEGATIVE', 'POSITIVE')
    new_df['date_new'] = pd.to_datetime(new_df['year'],format='%Y')
    new_df = new_df.sort_values(by='date_new')
    
    return new_df
    

In [ ]:
#plot graph for country df
def sentiment_graph(df, country):
    fig = go.Figure(
    data=go.Heatmap(
        z=df['Overall_Sentiment_Score'],
        x=df['year'],
        y=df["new_sentiment_score"],
        colorscale=px.colors.sequential.RdBu,
            )
        )

    fig.update_layout(
        title=go.layout.Title(
            text=f'Sentiment Analysis for {country}'
        ),
        autosize=False,
        width=1150,
        height=400,
    )

    fig.update_layout(yaxis_autorange = "reversed")
    return fig


## Testing function

In [51]:
k =get_df('PAK')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
k.to_csv('pakistan_sentiment.csv')

In [52]:
sentiment_graph(k,'Pakistan')

In [ ]:
pak

# Iran

In [122]:
df2[df['country']=='IND']

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/3898611169.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,session,year,country,text
247,68,2013,IND,"Let me first of all congratulate \nyou, Mr. Pr..."
565,63,2008,IND,"May I first congratulate you, \nSir, on your e..."
835,46,1991,IND,"﻿I have great pleasure in congratulating you, ..."
1057,57,2002,IND,﻿I\ncongratulate Mr. Kavan on his election as ...
1513,51,1996,IND,﻿It gives me great pleasure to\ncongratulate A...
1576,67,2012,IND,﻿It is indeed a great privilege\nfor me to be ...
1828,52,1997,IND,"﻿Mr. President, it is a particular\npleasure f..."
2227,65,2010,IND,"Allow me to begin by \ncongratulating you, Sir..."
2652,50,1995,IND,It gives me great pleasure to\ncongratulate Mr...
2869,64,2009,IND,I offer my congratulations \nto Mr. Treki on h...


In [11]:
iran.head()

,session,year,country,text
336,68,2013,IRN,"Mr. President, at the outset, I \nwould like t..."
663,63,2008,IRN,I am grateful to the \nAlmighty for granting m...
839,46,1991,IRN,"﻿At the outset, Sir, I should like to express ..."
1137,57,2002,IRN,﻿I\nwould like at the outset to congratulate M...
1454,51,1996,IRN,﻿It is a\nsource of profound satisfaction to s...


In [12]:
iran['Overall_Sentiment_Score'] = iran['text'].apply(sentiment_analysis)

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/2824453655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iran['Overall_Sentiment_Score'] = iran['text'].apply(sentiment_analysis)


In [13]:
iran = pd.DataFrame(iran)

In [14]:
iran['new_sentiment_score'] = np.where(iran['Overall_Sentiment_Score']< 0.3, 'NEGATIVE', 'POSITIVE')
iran.head()

,session,year,country,text,Overall_Sentiment_Score,new_sentiment_score
336,68,2013,IRN,"Mr. President, at the outset, I \nwould like t...",0.002566,NEGATIVE
663,63,2008,IRN,I am grateful to the \nAlmighty for granting m...,0.173459,NEGATIVE
839,46,1991,IRN,"﻿At the outset, Sir, I should like to express ...",0.498799,POSITIVE
1137,57,2002,IRN,﻿I\nwould like at the outset to congratulate M...,0.573124,POSITIVE
1454,51,1996,IRN,﻿It is a\nsource of profound satisfaction to s...,-0.035528,NEGATIVE


In [41]:
iran['date_new'] = pd.to_datetime(iran['year'],format='%Y')
iran = iran.sort_values(by='date_new')

In [42]:
iran.to_csv('iran_sentiment_analysis.csv')

In [44]:
sentiment_graph(iran, 'Iran')

In [39]:
fig = go.Figure(
    data=go.Heatmap(
        z=iran['Overall_Sentiment_Score'],
        x=iran['year'],
        y=iran["new_sentiment_score"],
        colorscale=px.colors.sequential.RdBu,
    )
)

fig.update_layout(
    title=go.layout.Title(
        text="Sentiment Analysis"
    ),
    autosize=False,
    width=1150,
    height=400,
)

fig.update_layout(yaxis_autorange = "reversed")

fig.show()

In [38]:
px.line(iran.sort_values(by='date_new'), x="date_new", y="Overall_Sentiment_Score")

# Doing Df for each country

## USA

In [55]:
usa =get_df('USA')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [56]:
usa.to_csv('usa_sentiment.csv')

In [57]:
sentiment_graph(usa, 'USA')

## Libiya

In [61]:
libiya =get_df('LBY')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [119]:
libiya.to_csv('libya_sentiment.csv')

In [62]:
sentiment_graph(libiya, 'Libya')

## Thai

In [74]:
thai = get_df('THA')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [114]:
thai.head()

,session,year,country,text,Overall_Sentiment_Score,new_sentiment_score,date_new
6481,45,1990,THA,"﻿On behalf of the Thai delegation, I should li...",0.663195,POSITIVE,1990-01-01
897,46,1991,THA,﻿I should like to extend warm congratulations ...,0.764193,POSITIVE,1991-01-01
5696,48,1993,THA,On behalf of the\nGovernment and the people of...,0.871516,POSITIVE,1993-01-01
6644,49,1994,THA,On behalf of the\nGovernment and people of Tha...,0.819994,POSITIVE,1994-01-01
2635,50,1995,THA,On behalf of the\nGovernment and people of Tha...,0.499902,POSITIVE,1995-01-01


In [76]:
sentiment_graph(thai, 'Thailand') #nah

## Swiss

In [69]:
swiss = pd.read_csv('/Users/yasmeenmahomedy2/code/yasmeeenm/conflict_prediction/yasmeen_folder/switzerland_sentiment.csv')

In [70]:
swiss

,Unnamed: 0,session,year,country,text,Overall_Sentiment_Score,new_sentiment_score,date_new
0,1077,57,2002,CHE,"﻿It is\nimperative, now more than ever before,...",0.604103,POSITIVE,2002-01-01
1,4374,58,2003,CHE,"﻿In 2000, at the dawn of\nthe new millennium, ...",0.109199,NEGATIVE,2003-01-01
2,4515,59,2004,CHE,Two years\nago I had the great satisfaction of...,0.126415,NEGATIVE,2004-01-01
3,3805,60,2005,CHE,"I would like to congratulate you,\nMr. Eliasso...",0.346218,POSITIVE,2005-01-01
4,7013,61,2006,CHE,We \nrecently celebrated the fiftieth annivers...,0.194725,NEGATIVE,2006-01-01
5,7293,62,2007,CHE,"At the \noutset, Sir, I should like to congra...",0.445252,POSITIVE,2007-01-01
6,581,63,2008,CHE,I \nhave the honour to speak on behalf of the ...,0.299479,NEGATIVE,2008-01-01
7,2849,64,2009,CHE,We look back \non a year that has been excepti...,0.492700,POSITIVE,2009-01-01
8,2361,65,2010,CHE,"It is a \ngreat honour for me to address, on b...",0.258604,NEGATIVE,2010-01-01
9,5096,66,2011,CHE,How \ncan we shape development policy that is ...,0.545266,POSITIVE,2011-01-01


In [115]:
swiss['new_sentiment_score'] = np.where(swiss['Overall_Sentiment_Score']<0, 'NEGATIVE', 'POSITIVE')


In [116]:
sentiment_graph(swiss, 'Switzerland') #its okS

## UK

In [102]:
uk = pd.read_csv('/Users/yasmeenmahomedy2/code/yasmeeenm/conflict_prediction/yasmeen_folder/uk_sentiment.csv')

In [103]:
sentiment_graph(uk, 'United Kingdom')

In [104]:
uk['new_sentiment_score'] = np.where(uk['Overall_Sentiment_Score']<= 0.1, 'NEGATIVE', 'POSITIVE')


In [109]:
uk.to_csv('uk_sentiment.csv')

In [108]:
fig = go.Figure(
    data=go.Heatmap(
        z=uk['Overall_Sentiment_Score'],
        x=uk['year'],
        y=uk["new_sentiment_score"],
        colorscale=px.colors.sequential.RdBu,
            )
        )

fig.update_layout(
        title=go.layout.Title(
            text='Sentiment Analysis for United Kingdom'
        ),
        autosize=False,
        width=1150,
        height=400,
    )

fig.update_layout(yaxis_autorange = "reversed")

## USA

In [99]:
usa = pd.read_csv('/Users/yasmeenmahomedy2/code/yasmeeenm/conflict_prediction/yasmeen_folder/usa_sentiment.csv')

In [100]:
sentiment_graph(usa, 'USA')  #maybe not?

In [110]:
ivory = pd.read_csv('/Users/yasmeenmahomedy2/code/yasmeeenm/conflict_prediction/yasmeen_folder/ivory_coast_sentiment.csv')

## Ivory

In [111]:
ivory.head()

,Unnamed: 0,session,year,country,text,Overall_Sentiment_Score,new_sentiment_score,date_new
0,904,46,1991,CIV,﻿The election of Ambassador Samir Shihabi to t...,0.441616,POSITIVE,1991-01-01
1,6196,47,1992,CIV,We grieved \nto learn of the aviation disaster...,0.285073,NEGATIVE,1992-01-01
2,5565,48,1993,CIV,I should like to pay tribute to the wisdom of ...,0.426273,POSITIVE,1993-01-01
3,6706,49,1994,CIV,Each\nyear’s session of the General Assembly p...,0.481126,POSITIVE,1994-01-01
4,1484,51,1996,CIV,"﻿My delegation and I are pleased, Sir, to see ...",0.509562,POSITIVE,1996-01-01


In [112]:
sentiment_graph(ivory, "Ivory Coast") #keeper!!!

## Sweden

In [118]:
sweden = get_df('SWE')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [128]:
sweden.to_csv('sweden_sentiment.csv')

In [120]:
sentiment_graph(sweden, 'Sweden')

## Indonesia

In [123]:
indonesia = get_df('IND')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [124]:
sentiment_graph(indonesia, 'Indonesia')

In [129]:
indonesia.to_csv('indonesia_sentiment.csv')

##  Sudan

In [127]:
 sudan = get_df('SDN')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [130]:
sentiment_graph(sudan, 'Sudan')

In [131]:
sudan.to_csv('sudan_sentiment.csv')

#  iraq

In [133]:
iraq = get_df('IRQ')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [134]:
sentiment_graph(iraq, 'Iraq')

In [135]:
iraq.to_csv('iraq_sentiment.csv')

## Afghanisatan

In [140]:
afg.to_csv('afg_sentiment.csv')

In [138]:
afg = get_df('AFG')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [139]:
sentiment_graph(afg, 'Afghanistan')

In [142]:
somalia = get_df('SOM')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [144]:
sentiment_graph(somalia , 'Somalia')

In [145]:
somalia.to_csv('somalia_sentiment.csv')

## Greece

In [149]:
greece = get_df("GRC")

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [152]:
sentiment_graph(greece, 'Greece')

## Turkey

In [161]:
turkey = get_df('TUR')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
sentiment_graph(turkey, 'Turkey')

In [172]:
turkey.to_csv('turkey_sentiment.csv')

## Brazil

In [169]:
brazil = get_df('BRA')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
sentiment_graph(brazil,"Brazil")

In [173]:
brazil.to_csv('brazil_sentiment.csv')

## Egypt

In [174]:
egypt = get_df('EGY')

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/ty/xzkrlw3n681fsxzmh9cl7wq00000gq/T/ipykernel_64340/1688464185.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [175]:
sentiment_graph(egypt , 'Egypt')

In [176]:
egypt.to_csv('egypt_sentiment.csv')